In [25]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from PIL import Image
import time
import matplotlib.pyplot as plt

In [26]:
BATCH_SIZE = 64
EPOCHS = 10
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pipeline = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

train_dataset = datasets.MNIST(root='./dataset', train=True, download=True, transform=pipeline)
test_dataset = datasets.MNIST(root='./dataset', train=False, download=True, transform=pipeline)
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [27]:
# image, target = train_dataset[0]
# print(image.shape)
#
# plt.imshow(image.reshape(28, 28), cmap='gray')
# print(target)

### Build Model

In [28]:
class MnistFC(nn.Module):
    def __init__(self):
        super(MnistFC, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=784, out_features=200),
            nn.ReLU(),
            nn.Linear(in_features=200, out_features=50),
            nn.ReLU(),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [29]:
model = MnistFC().to(device=DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0005)
loss_func = nn.CrossEntropyLoss()

### Training

In [30]:
def train_model(model, device, train_loader, optimizer, loss_function, epoch):
    # print('Start Training...')
    model.train()
    for batch_idx, (image, target) in enumerate(train_loader):
        image, target = image.to(device=device), target.to(device=device)
        optimizer.zero_grad()
        output = model(image)
        train_loss = loss_function(output, target)
        train_loss.backward()
        optimizer.step()

        if batch_idx % 3000 == 0:
            print("Train Round :{}\tLoss :{:.6f}".format(epoch, train_loss.item()))

### Testing

In [31]:
def test_model(model, device, test_loader, loss_function):
    # print('Start Testing...')
    model.eval()
    accuracy = 0.0
    test_loss = 0.0

    with torch.no_grad():
        for image, target in test_loader:
            image, target = image.to(device=device), target.to(device=device)
            output = model(image)
            test_loss += loss_function(output, target)
            pred = output.argmax(dim=1)  # tensor([8, 0, 0, 1, 0, 7, 8, 1, 8, 2, 3, 0, 2, 1, 1, 0, 4, 5, 6, 8, 0, 1, 2, 3, 1, 5, 2, 1, 1, 5, 4, 6, 3, 6, 5, 0, 5, 2, 2, 3, 9, 6, 1, 4, 9, 5, 5, 7, 3, 6, 0, 5, 2, 8, 3, 3, 4, 7, 1, 0, 5, 7, 3, 6], device='cuda:0')
            accuracy += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        accuracy /= len(test_loader.dataset)
        print("Average Test Loss :{:.4f}, Accuracy :{:.2f}%\n".format(test_loss.item(), accuracy * 100.0))

### Run

In [32]:
start_time = time.time()
for epoch in range(1, EPOCHS + 1):
    train_model(model=model, device=DEVICE, train_loader=train_loader, optimizer=optimizer, loss_function=loss_func, epoch=epoch)
    test_model(model=model, device=DEVICE, test_loader=test_loader, loss_function=loss_func)
end_time = time.time()
print('>> Total time:', end_time - start_time)

Train Round :1	Loss :3.913208
Average Test Loss :0.0531, Accuracy :57.11%

Train Round :2	Loss :3.278773
Average Test Loss :0.0530, Accuracy :57.44%

Train Round :3	Loss :3.445760
Average Test Loss :0.0529, Accuracy :57.79%

Train Round :4	Loss :3.311575
Average Test Loss :0.0529, Accuracy :57.97%

Train Round :5	Loss :3.414396
Average Test Loss :0.0528, Accuracy :58.12%

Train Round :6	Loss :3.342644
Average Test Loss :0.0528, Accuracy :58.30%

Train Round :7	Loss :3.316772
Average Test Loss :0.0528, Accuracy :58.43%

Train Round :8	Loss :3.368713
Average Test Loss :0.0528, Accuracy :58.52%

Train Round :9	Loss :3.345406
Average Test Loss :0.0528, Accuracy :58.56%

Train Round :10	Loss :3.319569
Average Test Loss :0.0528, Accuracy :58.68%

>> Total time: 58.09074354171753
